# Evidently Dashboard Prep

In [1]:
import os
import sys
import cdsw
import pickle
import sklearn
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset

# Install cmlapi package
try:
    import cmlapi
except ModuleNotFoundError:
    cluster = os.getenv("CDSW_API_URL")[:-1] + "2"
    !pip3 install {cluster}/python.tar.gz
    import cmlapi

from utils.utils import get_latest_deployment_details
from utils.inference_utils import ThreadedModelRequest, cast_date_as_str_for_json

%load_ext autoreload
%autoreload 2
%load_ext lab_black

## Simulation

### 1. Score Train Data

In [2]:
# format train data for inference
train_df = pd.read_pickle("../data/working/train_df.pkl")
train_records = cast_date_as_str_for_json(train_df).to_dict(orient="records")

In [9]:
# obtain model deployment info
client = cmlapi.default_client()
latest_deployment_details = get_latest_deployment_details(
    client=client, model_name="Price Regressor3"
)

In [10]:
latest_deployment_details

{'model_name': 'Price Regressor3',
 'model_id': 'bed9d7d6-a26f-4d32-964c-b2f6b48443db',
 'model_crn': 'crn:cdp:ml:us-west-1:12a0079b-1591-4ca0-b721-a446bda74e67:workspace:1e08299d-97ac-4d5e-8c21-d77745ce0a1c/bed9d7d6-a26f-4d32-964c-b2f6b48443db',
 'model_access_key': 'm4vghrhbcpdn112u7ta5ar5c9obnh3px',
 'latest_build_id': '96322a3e-40ac-4d98-a745-df626139e5b3',
 'latest_deployment_crn': 'crn:cdp:ml:us-west-1:12a0079b-1591-4ca0-b721-a446bda74e67:workspace:1e08299d-97ac-4d5e-8c21-d77745ce0a1c/018226e1-aa2e-4721-9801-01e6f5e9394f'}

In [12]:
%%time
tmr = ThreadedModelRequest(deployment_details=latest_deployment_details)
train_inference_metadata = tmr.threaded_call(train_records)

CPU times: user 52.9 s, sys: 1.01 s, total: 53.9 s
Wall time: 8min 33s


In [13]:
with open("train_inference_metadata.pkl", "wb") as f:
    pickle.dump(train_inference_metadata, f)

In [7]:
# with open("train_inference_metadata.pkl", "rb") as f:
#     train_inference_metadata = pickle.load(f)

#### Create master id <--> uuid mapping: this gets populated as soon as a property is "listed"

In [14]:
master_id_uuid_mapping = {}
master_id_uuid_mapping.update(train_inference_metadata["id_uuid_mapping"])

In [15]:
len(master_id_uuid_mapping)

11518

In [9]:
len(master_id_uuid_mapping)

11520

### 1.a Add delayed metrics

In [16]:
def add_delayed_metrics(uuids, ground_truths, sold_dates):
    """
    Add delayed metrics to CML Model Metrics database provided a
    list of prediction UUID's and corresponding list of ground truth values.

    """

    if len(uuids) != len(ground_truths) != len(sold_dates):
        raise ValueError(
            "UUIDs, ground_truths, and sold_dates must be of same length and correspond by index."
        )

    for uuid, gt, ds in zip(uuids, ground_truths, sold_dates):
        cdsw.track_delayed_metrics(
            metrics={"ground_truth": gt, "date_sold": ds}, prediction_uuid=uuid
        )

    print(f"Sucessfully added ground truth values to {len(uuids)} records.")

In [52]:
train_df[2:].id.apply(lambda x: master_id_uuid_mapping[x]).tolist()

['d624f91d-71d6-4e2f-83f7-97052a90ad98',
 'c3360eb7-d031-40f8-bca7-5fe82df9bad9',
 'b761eaec-860a-418c-abdd-d096c9274038',
 'a1a2ec46-6802-4e08-af14-9775463d80a8',
 'c7876539-9e4d-4563-8ee1-b41977d91c7f',
 'ce8dee4d-25a7-4630-95f3-792e6210d8e8',
 '44ae35ec-1163-494e-aec0-0cb168c8c4fb',
 'eede32ce-9e7d-46a4-bb18-10f75fcaa83c',
 '3e5eb624-f7d1-4376-9882-6655d736dd36',
 'e40c7e01-94b5-4382-bf8c-95776ed5c44c',
 'c3d1687f-81c2-4899-8393-ac518e8f0822',
 'b3737d8d-85db-4da4-860d-3c8986a9841b',
 '312abc53-376d-4345-ac19-26a8e8c2446d',
 '81bb1973-242f-43ae-852f-054af56adb3d',
 'fb42eb17-e460-4c64-8650-0e7b3c90ae01',
 'c46e263c-bde1-4408-8fdb-1bfb59ef8370',
 'e0b78973-59fb-4e4a-9b90-98541abbef0d',
 'e0643078-6a93-4a3f-bbdb-f94cfe44eaab',
 '9e034911-b8e4-48c9-996f-ca06b11cb146',
 '86d1e513-7643-477c-9d91-f8f2e84df847',
 '6b541af0-2438-4b6a-a0c0-fe530d3475c0',
 'c451cf75-a0c7-4730-b337-55c82dd2a127',
 'd2cf7bff-2219-454c-b8ec-ebf8ef21e3be',
 '88c50200-3e35-4aad-9df6-8bf25ab8e17b',
 'edd21e89-9e16-

In [50]:
train_df.shape

(11520, 22)

In [51]:
len(train_inference_metadata["id_uuid_mapping"])

11518

In [49]:
len(master_id_uuid_mapping)

15272

In [17]:
# get list of uuids from train_inference_metadata
train_uuids = list(train_inference_metadata["id_uuid_mapping"].values())

# get list of prices
train_gts = train_df[
    train_df.id.isin(train_inference_metadata["id_uuid_mapping"].keys())
].price.tolist()

# get list of sold_dates
train_sold_dates = train_df[
    train_df.id.isin(train_inference_metadata["id_uuid_mapping"].keys())
].date_sold.tolist()

In [18]:
%%time

add_delayed_metrics(train_uuids, train_gts, train_sold_dates)

Sucessfully added ground truth values to 11518 records.
CPU times: user 2min 14s, sys: 4.24 s, total: 2min 18s
Wall time: 4min 2s


### 1.b Query the metadata store

In [19]:
metrics = cdsw.read_metrics(
    model_deployment_crn=latest_deployment_details["latest_deployment_crn"],
    start_timestamp_ms=train_inference_metadata["start_timestamp_ms"],
    end_timestamp_ms=train_inference_metadata["end_timestamp_ms"],
)

In [20]:
from typing import Dict


def format_model_metrics_query(metrics: Dict):
    """
    Accepts the response dictionary from `cdsw.read_metrics()`, filters out any non-metrics columns,
    and formats as Dataframe.
    """
    metrics = pd.json_normalize(metrics["metrics"])

    return metrics[
        [col for col in metrics.columns if col.split(".")[0] == "metrics"]
        + ["predictionUuid"]
    ].rename(columns={col: col.split(".")[-1] for col in metrics.columns})

In [21]:
train_metrics_df = format_model_metrics_query(metrics)

In [22]:
train_metrics_df

,date_sold,ground_truth,view,zipcode,bedrooms,sqft_lot,bathrooms,condition,sqft_above,waterfront,sqft_living,predicted_result,predictionUuid
0,2014-05-02,285000.0,0,98003,3,10834,2.50,4,1360,0,2090,2.879715e+05,d624f91d-71d6-4e2f-83f7-97052a90ad98
1,2014-05-02,287200.0,0,98038,3,19966,3.00,4,1090,0,1850,2.997052e+05,c3360eb7-d031-40f8-bca7-5fe82df9bad9
2,2014-05-02,435000.0,0,98006,4,8800,1.00,4,1450,0,1450,4.538114e+05,a1a2ec46-6802-4e08-af14-9775463d80a8
3,2014-05-02,550000.0,0,98052,4,10500,2.50,4,1140,0,1940,5.242286e+05,c7876539-9e4d-4563-8ee1-b41977d91c7f
4,2014-05-02,630000.0,0,98027,4,37277,2.75,3,2710,0,2710,6.291010e+05,ce8dee4d-25a7-4630-95f3-792e6210d8e8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11513,2014-10-31,565000.0,0,98056,4,10016,2.50,3,2620,0,2620,4.686644e+05,e75e0938-8fc6-4587-87f2-a7519b5926e7
11514,2014-10-31,903000.0,0,98006,3,12786,2.75,4,3860,0,3860,1.113453e+06,5b3bba9f-d6b5-4b3f-bb3e-663cdb055e65
11515,2014-10-31,575000.0,0,98034,3,1559,3.50,3,2024,0,2514,5.385518e+05,4163f543-c9ee-4d47-9c2a-7f59363ea647
11516,2014-10-31,635000.0,0,98004,3,7482,2.25,4,1240,0,1940,8.172523e+05,cd26bf4f-9d63-4beb-8adc-7e5e623d3fe4


### 2. Create Simulation

In [23]:
prod_path = "../data/working/prod_df.pkl"
prod_df = pd.read_pickle(prod_path)

In [24]:
# number of months in prod set
n_months = int(
    np.ceil(
        (prod_df.date_sold.max() - prod_df.date_sold.min()) / np.timedelta64(1, "M")
    )
)

In [25]:
# construct date ranges to iterate through as simulation of time (include left, exclude right)

date_ranges = [
    [
        (prod_df.date_sold.min() + DateOffset(months=n)),
        (prod_df.date_sold.min() + DateOffset(months=n + 2)),
    ]
    for n in range(0, n_months, 2)
]

# increase first date range to account for records that listed during the train_df timeframe
date_ranges[0][0] = date_ranges[0][0] - DateOffset(years=1)

In [26]:
date_ranges

[[Timestamp('2013-10-31 00:00:00'), Timestamp('2014-12-31 00:00:00')],
 [Timestamp('2014-12-31 00:00:00'), Timestamp('2015-02-28 00:00:00')],
 [Timestamp('2015-02-28 00:00:00'), Timestamp('2015-04-30 00:00:00')],
 [Timestamp('2015-04-30 00:00:00'), Timestamp('2015-06-30 00:00:00')]]

### Logic for one loop

#### Query prod_df for new listings in new date range + make inference

In [27]:
new_listings_records = prod_df.loc[
    prod_df.date_listed.between(date_ranges[0][0], date_ranges[0][1], inclusive="left")
]

new_listings_records = cast_date_as_str_for_json(new_listings_records).to_dict(
    orient="records"
)

/home/cdsw/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:
%%time
tmr = ThreadedModelRequest(deployment_details=latest_deployment_details)
new_listing_inference_metadata = tmr.threaded_call(new_listings_records)

In [ ]:
len(new_listing_inference_metadata["id_uuid_mapping"])

In [ ]:
master_id_uuid_mapping.update(new_listing_inference_metadata["id_uuid_mapping"])

In [ ]:
len(master_id_uuid_mapping)

In [ ]:
with open("master_id_uuid_mapping.pkl", "wb") as f:
    pickle.dump(master_id_uuid_mapping, f)

#### Query prod_df for newly sold properties in new date range + assign ground truth to records in metric store

In [ ]:
new_sold_records = prod_df.loc[
    prod_df.date_sold.between(date_ranges[0][0], date_ranges[0][1], inclusive="left")
]

In [ ]:
# get list of uuids from 
new_sold_uuids = new_sold_records.id.apply(lambda x: master_id_uuid_mapping[x]).tolist()

# get list of ground truth prices for newly sold properties
new_sold_gts = prod_df[prod_df.id.isin(new_sold_records.id)].price.tolist()

# get list of sold_dates for newly sold properties
new_sold_dates = (
    prod_df[prod_df.id.isin(new_sold_records.id)].date_sold.astype(str).tolist()
)

In [ ]:
%%time
add_delayed_metrics(new_sold_uuids, new_sold_gts, new_sold_dates)

#### Query metric store for newly sold records for evidently reporting

In [39]:
# because I cant query by UUID, I must query all records, then filter to new_sold by uuid
metrics = cdsw.read_metrics(
    model_deployment_crn=latest_deployment_details["latest_deployment_crn"]
)

metrics_df = format_model_metrics_query(metrics)

In [40]:
new_sold_metrics_df = metrics_df[metrics_df.predictionUuid.isin(new_sold_uuids)]

## Build Reporting Dashboard

In [42]:
from evidently.dashboard import Dashboard
from evidently.tabs import DataDriftTab, NumTargetDriftTab, RegressionPerformanceTab

In [43]:
tm = "train_metrics_df.pkl"
sm = "new_sold_metrics_df.pkl"

train_metrics_df.to_pickle(tm)
new_sold_metrics_df.to_pickle(sm)

train_metrics_df = pd.read_pickle(tm)
new_sold_metrics_df = pd.read_pickle(sm)

In [44]:
def scale_prices(df):
    """
    Scale prices from being denominated in dollars to hundreds of thousands of dollars.
    """
    copy = df.copy(deep=True)
    for col in ("ground_truth", "predicted_result"):
        copy[col] = copy[col] / 100_000

    return copy

In [45]:
TARGET = "ground_truth"
PREDICTION = "predicted_result"
NUM_FEATURES = ["sqft_living", "sqft_lot", "sqft_above"]
CAT_FEATURES = ["waterfront", "zipcode", "condition", "view", "bedrooms", "bathrooms"]

column_map = {
    "target": TARGET,
    "prediction": PREDICTION,
    "numerical_features": NUM_FEATURES,
    "categorical_features": CAT_FEATURES,
    "datetime": None,
}

dashboard = Dashboard(tabs=[DataDriftTab, NumTargetDriftTab, RegressionPerformanceTab])

dashboard.calculate(
    reference_data=scale_prices(train_metrics_df)
    .sample(n=len(new_sold_metrics_df), random_state=42)
    .set_index("date_sold", drop=True)
    .sort_index()
    .round(2),
    current_data=scale_prices(new_sold_metrics_df)
    .set_index("date_sold", drop=True)
    .sort_index()
    .round(2),
    column_mapping=column_map,
)

dashboard.save("../apps/reports/price_regressor.html")

/home/cdsw/.local/lib/python3.6/site-packages/scipy/stats/stats.py:5982: RuntimeWarning:

divide by zero encountered in true_divide

